# Data Section

In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import folium

# Postal Code extraction

In [2]:
link = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
get_text = requests.get(link).text
xml = BeautifulSoup(get_text, 'xml')
table=xml.find('table')
columns = ['Postalcode','Borough','Neighborhood']
df = pd.DataFrame(columns = columns)

# Search for postcodes, borough and neighborhood 
for tr_cell in table.find_all('tr'):
    row_data=[]
    for td_cell in tr_cell.find_all('td'):
        row_data.append(td_cell.text.strip())
    if len(row_data)==3:
        df.loc[len(df)] = row_data

In [3]:
df.head(12)

,Postalcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
8,M8A,Not assigned,Not assigned
9,M9A,Etobicoke,Islington Avenue


# Data treatment

In [4]:
#1)Bring just rows where Borough is diferent from 'Not assigned'
df = df[df.Borough != 'Not assigned']

#2)If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.
for index, row in df.iterrows():
    if row['Neighborhood'] == 'Not assigned':
        row['Neighborhood'] = row['Borough']

#3)Grouping per postcode and bringing neighborhoods and boroughs to the same line  
df=df.groupby(['Postalcode','Borough'])['Neighborhood'].apply(','.join).reset_index()
df.head(15)


,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff,Cliffside West"


In [5]:
df.shape

(103, 3)

# Geting geo data

In [6]:
geo_data=pd.read_csv('http://cocl.us/Geospatial_data')

In [7]:
geo_data.head(10)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [8]:
geo_data.rename(columns={'Postal Code':'Postalcode'},inplace=True)
df_geo = pd.merge(geo_data, df, on='Postalcode')

In [9]:
# create map of New York using latitude and longitude values
map = folium.Map(location=[43.806686, -79.194353], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_geo['Latitude'], df_geo['Longitude'], df_geo['Borough'], df_geo['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map) 
map

## Lets create a fucntion to explore and bring data from the foursquare api

In [11]:
CLIENT_ID = '44GZRUGU0KV34BK1VJ0ED52T1MESILINW1ZH0OQ3HQY1P1K1' # your Foursquare ID
CLIENT_SECRET = 'CXORDLZIUGYFOAZQZXJNG2DYXMTMLF1DE041QPN3RW4MJ3LC' # your Foursquare Secret
VERSION = '20190425'
limit=1000000

In [12]:
# Let's create a function to repeat the process to all the neighborhoods in Toronto
def getNearbyVenues(names, latitudes,longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names,latitudes,longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [13]:
downtown_toronto_venues = getNearbyVenues(names=df_geo['Neighborhood'],latitudes=df_geo['Latitude'],longitudes=df_geo['Longitude'], radius=500)

Rouge,Malvern
Highland Creek,Rouge Hill,Port Union
Guildwood,Morningside,West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park,Ionview,Kennedy Park
Clairlea,Golden Mile,Oakridge
Cliffcrest,Cliffside,Scarborough Village West
Birch Cliff,Cliffside West
Dorset Park,Scarborough Town Centre,Wexford Heights
Maryvale,Wexford
Agincourt
Clarks Corners,Sullivan,Tam O'Shanter
Agincourt North,L'Amoreaux East,Milliken,Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview,Henry Farm,Oriole
Bayview Village
Silver Hills,York Mills
Newtonbrook,Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park,Don Mills South
Bathurst Manor,Downsview North,Wilson Heights
Northwood Park,York University
CFB Toronto,Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens,Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West,Riverdale
The Beaches West,Indi

# Now, we need to calculate the frequency of each type of venue in order to stablish clusters

In [14]:
downtown_toronto_venues['Venue Category2'] = downtown_toronto_venues['Venue Category'].str.contains(pat = 'Restaurant')
downtown_toronto_venues['Venue Category2']= downtown_toronto_venues.apply(lambda x: 'Restaurant' if x['Venue Category2'] else x['Venue Category'], axis=1)
downtown_toronto_venues['Venue Category']=downtown_toronto_venues['Venue Category2']

In [15]:
# one hot encoding
downtown_toronto_onehot = pd.get_dummies(downtown_toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
downtown_toronto_onehot['Neighborhood'] = downtown_toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [downtown_toronto_onehot.columns[-1]] + list(downtown_toronto_onehot.columns[:-1])
downtown_toronto_onehot = downtown_toronto_onehot[fixed_columns]

downtown_toronto_onehot.head() 

,Yoga Studio,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,Antique Shop,Aquarium,...,Toy / Game Store,Trail,Train Station,Video Game Store,Video Store,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
downtown_toronto_grouped = downtown_toronto_onehot.groupby('Neighborhood').mean().reset_index()

In [17]:
num_top_venues = 5

for hood in downtown_toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = downtown_toronto_grouped[downtown_toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
         venue  freq
0   Restaurant  0.31
1  Coffee Shop  0.07
2         Café  0.04
3          Bar  0.04
4   Steakhouse  0.03


----Agincourt----
                 venue  freq
0               Lounge  0.25
1         Skating Rink  0.25
2       Breakfast Spot  0.25
3           Restaurant  0.25
4  Monument / Landmark  0.00


----Agincourt North,L'Amoreaux East,Milliken,Steeles East----
                venue  freq
0                Park   0.5
1          Playground   0.5
2         Yoga Studio   0.0
3               Motel   0.0
4  Mac & Cheese Joint   0.0


----Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown----
                 venue  freq
0        Grocery Store  0.18
1           Restaurant  0.18
2             Pharmacy  0.09
3  Fried Chicken Joint  0.09
4          Pizza Place  0.09


----Alderwood,Long Branch----
          venue  freq
0   Pizza Place  0.22
1           Gym  0.11
2  Skating Rink  0.11
3   Coffee

           venue  freq
0            Gym  0.07
1         Bakery  0.07
2  Grocery Store  0.07
3     Kids Store  0.07
4    Flower Shop  0.07


----L'Amoreaux West----
           venue  freq
0     Restaurant  0.31
1    Coffee Shop  0.15
2    Pizza Place  0.08
3       Pharmacy  0.08
4  Grocery Store  0.08


----Lawrence Heights,Lawrence Manor----
                venue  freq
0      Clothing Store  0.25
1   Accessories Store  0.17
2          Restaurant  0.08
3         Event Space  0.08
4  Miscellaneous Shop  0.08


----Lawrence Park----
                 venue  freq
0                 Park  0.33
1          Swim School  0.33
2             Bus Line  0.33
3          Yoga Studio  0.00
4  Monument / Landmark  0.00


----Leaside----
                    venue  freq
0              Restaurant  0.09
1             Coffee Shop  0.09
2     Sporting Goods Shop  0.09
3            Burger Joint  0.06
4  Furniture / Home Store  0.06


----Little Portugal,Trinity----
         venue  freq
0   Restaurant  0.30
1   

In [18]:
# Let's put that into a pandas dataframe
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [19]:
import numpy as np
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = downtown_toronto_grouped['Neighborhood']

for ind in np.arange(downtown_toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(downtown_toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Restaurant,Coffee Shop,Bar,Café,Steakhouse,Clothing Store,Pizza Place,Hotel,Cosmetics Shop,Concert Hall
1,Agincourt,Restaurant,Skating Rink,Lounge,Breakfast Spot,Donut Shop,Distribution Center,Discount Store,Diner,Dessert Shop,Department Store
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",Park,Playground,Creperie,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner,Dessert Shop,Department Store
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",Grocery Store,Restaurant,Liquor Store,Beer Store,Discount Store,Pharmacy,Pizza Place,Sandwich Place,Fried Chicken Joint,Dance Studio
4,"Alderwood,Long Branch",Pizza Place,Gym,Pharmacy,Sandwich Place,Pub,Pool,Skating Rink,Coffee Shop,College Gym,College Rec Center
5,"Bathurst Manor,Downsview North,Wilson Heights",Restaurant,Coffee Shop,Bridal Shop,Ice Cream Shop,Supermarket,Diner,Pizza Place,Pharmacy,Fried Chicken Joint,Frozen Yogurt Shop
6,Bayview Village,Restaurant,Café,Bank,Women's Store,Cupcake Shop,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner
7,"Bedford Park,Lawrence Manor East",Restaurant,Coffee Shop,Sandwich Place,Pharmacy,Locksmith,Juice Bar,Pub,Café,Pizza Place,Butcher
8,Berczy Park,Restaurant,Coffee Shop,Cheese Shop,Beer Bar,Cocktail Bar,Bakery,Café,Farmers Market,Jazz Club,Beach
9,"Birch Cliff,Cliffside West",Skating Rink,General Entertainment,College Stadium,Café,Women's Store,Cupcake Shop,Dog Run,Distribution Center,Discount Store,Diner


## Let's put each neighborhood into a cluster to find out what is its the predominant characterisc and specially what regions are more correlated to restaurants

In [20]:
from sklearn.cluster import KMeans
import numpy as np


kclusters = 10

downtown_toronto_grouped_clustering = downtown_toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(downtown_toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
downtown_toronto_grouped_clustering['cluster']=pd.DataFrame(kmeans.labels_) 



In [21]:

#cluster vs neighboordhoods
clusters=pd.DataFrame([])
clusters['cluster']=downtown_toronto_grouped_clustering['cluster']
clusters['Neighborhood']=downtown_toronto_grouped['Neighborhood']
neighborhoods_venues_sorted=neighborhoods_venues_sorted.merge(clusters,right_on='Neighborhood',left_on='Neighborhood',how='left')

In [22]:
cluster_group=neighborhoods_venues_sorted.groupby(['cluster','1st Most Common Venue']).agg({'1st Most Common Venue': "count"})
cluster_group

1st Most Common Venue
cluster 1st Most Common Venue                       
0       Coffee Shop                                1
        Motel                                      1
        Pizza Place                                1
        Restaurant                                11
1       Airport Service                            1
        Bakery                                     2
        Beer Store                                 2
        Bus Line                                   1
        Coffee Shop                                1
        Field                                      1
        Grocery Store                              2
        Gym                                        2
        Mobile Phone Shop                          1
        Park                                       7
        Pizza Place                                2
        Pool                                       1
2       Gym                                        1
3       Playground                                 2
4       Clothing Store                             1
        Coffee Shop                                3
        Grocery Store                              2
        Pizza Place                                1
        Restaurant                                39
        Skating Rink                               1
5       Baseball Field                             3
6       Drugstore                                  1
7       Park                                       5
8       Bar                                        1
9       Coffee Shop                                2
        Grocery Store                              1

## Cluster 4 is the restaurant cluster. Now we need to choose the best neighborhood for new restaurant. To do so, lets see where we have the smallest frequency of restaurants inside the cluster 4


In [32]:
restaurants_freq=downtown_toronto_grouped[['Restaurant','Neighborhood']]
restaurant_cluster=neighborhoods_venues_sorted.loc[neighborhoods_venues_sorted['cluster']==4]
restaurant_cluster=restaurant_cluster.merge(restaurants_freq,right_on='Neighborhood',left_on='Neighborhood',how='left')
restaurant_cluster.sort_values(by=['Restaurant'],inplace=True)
restaurant_cluster.rename(columns={'Restaurant':'Restaurant frequency'},inplace=True)

In [35]:
restaurant_cluster

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,cluster,Restaurant frequency
5,"Birch Cliff,Cliffside West",Skating Rink,General Entertainment,College Stadium,Café,Women's Store,Cupcake Shop,Dog Run,Distribution Center,Discount Store,Diner,4,0.000000
46,"Woodbine Gardens,Parkview Hill",Pizza Place,Café,Pharmacy,Intersection,Bus Line,Restaurant,Bank,Athletics & Sports,Breakfast Spot,Gym / Fitness Center,4,0.083333
28,"Lawrence Heights,Lawrence Manor",Clothing Store,Accessories Store,Miscellaneous Shop,Furniture / Home Store,Coffee Shop,Boutique,Event Space,Restaurant,Athletics & Sports,Curling Ice,4,0.083333
29,Leaside,Coffee Shop,Sporting Goods Shop,Restaurant,Burger Joint,Furniture / Home Store,Gym,Sandwich Place,Fish & Chips Shop,Electronics Store,Beer Store,4,0.093750
7,Business Reply Mail Processing Centre 969 Eastern,Restaurant,Yoga Studio,Spa,Gym / Fitness Center,Garden Center,Garden,Farmers Market,Light Rail Station,Comic Shop,Pizza Place,4,0.111111
12,Christie,Grocery Store,Café,Park,Restaurant,Baby Store,Nightclub,Candy Store,Diner,Coffee Shop,Gas Station,4,0.117647
6,"Brockton,Exhibition Place,Parkdale Village",Restaurant,Coffee Shop,Café,Breakfast Spot,Gym,Bakery,Grocery Store,Furniture / Home Store,Intersection,Convenience Store,4,0.136364
23,Harbourfront,Coffee Shop,Restaurant,Bakery,Pub,Park,Theater,Breakfast Spot,Café,Cosmetics Shop,Shoe Store,4,0.140000
21,"Guildwood,Morningside,West Hill",Restaurant,Electronics Store,Spa,Breakfast Spot,Intersection,Medical Center,Rental Car Location,Cupcake Shop,Discount Store,Diner,4,0.142857
24,"Harbourfront East,Toronto Islands,Union Station",Restaurant,Coffee Shop,Aquarium,Café,Hotel,Fried Chicken Joint,Brewery,Scenic Lookout,Pizza Place,History Museum,4,0.150000


# Conclusion


### Inside the Restaurant Cluster, the neighborhood with lack of restaurants is the Birch Cliff,Cliffside West with the least frequency of restaurants. 
### According to https://en.wikipedia.org/wiki/Birch_Cliff. The Birch Cliff has a large Irish population. It has high sinergy to start a Irish restaurant arroud there. 